In [1]:
import os
import pandas as pd
import speech_recognition as sr
from fuzzywuzzy import fuzz
from gtts import gTTS
import tempfile
import threading
import time
import logging

# === SETTINGS === #
DATASET_AUDIO = "C:\\Users\\Hp\\Downloads\\aero"
CSV_PATH ="C:\\Users\\Hp\\Downloads\\voice_cockpit_dataset.xlsx"
CONFIRM_CRITICAL = ["autopilot_on", "autopilot_off"]
DEBUG_MODE = False

# === LOGGING === #
logging.basicConfig(level=logging.INFO if DEBUG_MODE else logging.WARNING)

# === CHECK DEPENDENCIES AND FILES === #
def check_dependencies():
    missing = []
    try:
        import speech_recognition
    except ImportError:
        missing.append("speech_recognition")
    try:
        import pyttsx3
    except ImportError:
        print("⚠️ pyttsx3 not available; using gTTS fallback.")
    try:
        import fuzzywuzzy
    except ImportError:
        missing.append("fuzzywuzzy")
    try:
        import gtts
    except ImportError:
        missing.append("gtts")
    if missing:
        print(f"❌ Missing dependencies: {', '.join(missing)}")
        print(f"Install with: pip install {' '.join(missing)}")
        exit(1)

def check_files():
    if not os.path.exists(CSV_PATH):
        print(f"❌ Excel file not found: {CSV_PATH}")
        exit(1)
    if not os.path.exists(DATASET_AUDIO):
        print(f"⚠️ Audio folder not found: {DATASET_AUDIO}")
        exit(1)

check_dependencies()
check_files()

# === LOAD EXCEL DATA === #
def load_data():
    try:
        data = pd.read_excel(CSV_PATH)
        if 'transcript' not in data.columns or 'intent' not in data.columns:
            print("❌ Excel file must contain 'transcript' and 'intent' columns.")
            exit(1)
        intent_map = dict(zip(data['transcript'], data['intent']))
        return intent_map
    except Exception as e:
        print(f"❌ Error loading Excel: {e}")
        exit(1)

intent_map = load_data()

# === ENGINE SETUP === #
recognizer = sr.Recognizer()
tts_engine = None
try:
    import pyttsx3
    tts_engine = pyttsx3.init()
    tts_engine.setProperty("rate", 150)
except Exception as e:
    logging.warning(f"pyttsx3 failed: {e}. Using gTTS instead.")

# === ACTION MAP === #
ACTIONS = {
    "autopilot_on": "🟢 Autopilot Activated",
    "autopilot_off": "🔴 Autopilot Deactivated",
    "increase_altitude": "⬆️ Increasing Altitude",
    "decrease_altitude": "⬇️ Decreasing Altitude",
}

# === SPEECH RECOGNITION === #
def recognize_audio_file(path, language="en-US"):
    if not os.path.exists(path):
        return None, "File not found."
    try:
        with sr.AudioFile(path) as source:
            audio = recognizer.record(source)
            text = recognizer.recognize_google(audio, language=language).lower()
            return text, None
    except sr.UnknownValueError:
        return None, "Speech not understood. Try clearer audio."
    except sr.RequestError as e:
        return None, f"Recognition service error: {e}"
    except Exception as e:
        return None, f"Unexpected error: {e}"

# === INTERPRET COMMAND === #
def interpret_command(command, threshold=80):
    if not command:
        return "unknown"
    best_match = None
    best_score = 0
    for word, intent in intent_map.items():
        score = fuzz.partial_ratio(word.lower(), command)
        if score > best_score and score >= threshold:
            best_match = intent
            best_score = score
    return best_match if best_match else "unknown"

# === PERFORM ACTION === #
def perform_action(intent):
    response = ACTIONS.get(intent, "⚠️ Command not recognized")
    print(f"Action: {response}")

    def speak(text):
        try:
            if tts_engine:
                tts_engine.say(text)
                tts_engine.runAndWait()
            else:
                tts = gTTS(text=text, lang='en')
                with tempfile.NamedTemporaryFile(delete=False, suffix='.mp3') as f:
                    tts.save(f.name)
                    os.system(f"start {f.name}" if os.name == 'nt' else f"afplay {f.name}")
                    time.sleep(2)
                    os.unlink(f.name)
        except Exception as e:
            logging.warning(f"TTS failed: {e}")
            print("⚠️ Voice output failed.")

    threading.Thread(target=speak, args=(response,), daemon=True).start()
    return response

# === MAIN EXECUTION === #
if __name__ == "__main__":
    print("\n🛫 Smart Voice-Activated Cockpit Assistant")
    print("Listening to all .wav commands in dataset...\n")

    try:
        files = [f for f in os.listdir(DATASET_AUDIO) if f.lower().endswith(".wav")]
        if not files:
            print("⚠️ No .wav files found in dataset.")
            exit(0)

        for f in files:
            audio_path = os.path.join(DATASET_AUDIO, f)
            print(f"🎧 Processing: {f}")
            cmd, error = recognize_audio_file(audio_path)

            if error:
                print(f"❌ {error}")
            elif cmd:
                print(f"🗣️ Recognized: '{cmd}'")
                intent = interpret_command(cmd)
                if intent in CONFIRM_CRITICAL:
                    confirm = input(f"Confirm action '{intent}'? (y/n): ").lower()
                    if confirm == 'y':
                        perform_action(intent)
                    else:
                        print("❎ Action cancelled.")
                else:
                    perform_action(intent)
            else:
                print("❌ Could not recognize this file.")
            print("---")

    except Exception as e:
        print(f"❌ Error: {e}")

    print("✅ Done. Simulation complete.")


ModuleNotFoundError: No module named 'speech_recognition'